In [ ]:
# Funkcja odpalana jako pandas_udf




In [ ]:
from abc import ABC, abstractmethod
from typing import List



class RetrainingStrategy(ABC):
    
    @abstractmethod
    def get_retraining_data(x_history: List, y_history: List[int], prediction_history: List[int], drift_history: List[int]):
        pass

In [ ]:
from abc import ABC, abstractmethod
from typing import List


class EvaluationStrategyManager(ABC):
    
    @abstractmethod
    def get_curr_ref_data(self, x_history: List, y_history: List[int], prediction_history: List[int], drift_history: List[int]):
        pass

In [ ]:
from abc import ABC, abstractmethod
from typing import List


class ModelEstimatorPipeline(ABC):
    
    @abstractmethod
    def handle(self, x, y):
        pass

    @abstractmethod
    def adjust_model(self, x_history: List, y_history: List[int], prediction_history: List[int], drift_history: List[int]):
        pass

    @abstractmethod
    def get_name(self):
        pass

In [ ]:
from sklearn.pipeline import Pipeline
from typing import Dict, List



class ModelSklearnPipeline(ModelEstimatorPipeline):

    def __init__(self, sklearn_pipeline: Pipeline, hyperparameter_space: Dict, retraining_strategy: TrainingStrategyManager):
        self.estimator = sklearn_pipeline
        self.hyperparameter_space = hyperparameter_space
        self.retraining_strategy = retraining_strategy

    def handle(self, x, y):
        return self.estimator.predict(x)
    
    def adjust_model(self, x_history: List, y_history: List[int], prediction_history: List[int], drift_history: List[int]):
        x_train, y_train = self.retraining_strategy.get_retraining_data(x_history, y_history, prediction_history, drift_history)
        self.estimator.fit(x_train, y_train) 

    def get_name(self):
        return super().get_name() # TO DO

In [ ]:
from river import compose
from typing import List


class ModelRiverPipeline(ModelEstimatorPipeline):
    
    def __init__(self, river_pipeline: compose.Pipeline):
        self.estimator = river_pipeline

    def handle(self, x, y):
        prediction = self.estimator.predict_one(x)
        self.estimator.learn_one(x, y)
        return prediction

    def adjust_model(self, x_history: List, y_history: List[int], prediction_history: List[int], drift_history: List[int]):
        return
    
    def get_name(self):
        return super().get_name() # TO DO

In [ ]:
class ModelEvaluationPipeline:
    
    def __init__(self, metric_steps):
        self.metric_steps = metric_steps

    def handle(self, y_true, y_predict):
        results = {}
        for metric_name, metric in self.steps:
            metric_value = metric.update(y_true, y_predict)
            results.update({metric_name: metric_value})
        return results

In [ ]:
from abc import ABC, abstractmethod
from typing import List



class MonitoringStep(ABC):

    @abstractmethod
    def monitor(self, x_history: List, y_history: List[int], prediction_history: List[int], drift_history: List[int]) -> bool:
        pass

In [ ]:
from evidently.test_suite import TestSuite



class EvidentlyMonitoringStep(MonitoringStep):

    def __init__(self, evidently_test_suite: TestSuite, evaluation_strategy: EvaluationStrategyManager):
        self.detector = evidently_test_suite
        self.eval_strategy = evaluation_strategy

    def monitor(self, x_history: List, y_history: List[int], prediction_history: List[int], drift_history: List[int]) -> bool:
        curr, ref = self.eval_strategy.get_curr_ref_data(x_history, y_history, prediction_history, drift_history)
        self.detector.run(reference_data=ref,current_data=curr)
        report = self.detector.as_dict()
        return True # to do based on report
        

In [ ]:
# Detector

    ## Monitoring - when?
        #[Adwin]
        #[EvidentlyAI TestSuite]



    ## Retraining Strategy - which retraining data?



        

In [1]:
import pandas as pd
from evidently.test_suite import TestSuite
from evidently.test_preset import DataDriftTestPreset
from evidently.tests import *
from batchstream.history.base.history_manager import HistoryManager
from batchstream.monitoring.pipeline.model_monitoring_pipeline import ModelMonitoringPipeline
from batchstream.monitoring.pipeline.steps.batch.evidently_monitoring_step import EvidentlyMonitoringStep
from sklearn.datasets import load_breast_cancer
from batchstream.utils.logging.base.logger_factory import LoggerFactory
from batchstream.batch_monitoring_strategy.dummy_monitoring_strategy import DummyMonitoringStrategy
from batchstream.retraining_strategy.dummy_retraining_strategy import DummyRetrainingStrategy 
from batchstream.model_comparers.batch_comparer import BatchModelComparer
from batchstream.model_comparers.shadow_comparer import ShadowOnlineComparer
from batchstream.pipelines.batch.batch_pipeline import BatchPipeline
from batchstream.estimators.sklearn_estimator import SklearnEstimator
from batchstream.detectors.base.detector import DriftDetector
from batchstream.experiment.experiment import StreamExperiment
from river.metrics import Accuracy, ROCAUC
from batchstream.evaluation.model_evaluation_pipeline import ModelEvaluationPipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline



history = HistoryManager()
logger_factory = LoggerFactory('test-2220')


### INPUT DRIFT DETECTION
# Detector 1.1 - Data Drift
data_drift_test_suite = TestSuite(tests=[
   DataDriftTestPreset(),
])
d1 = DummyMonitoringStrategy(n_curr=120, n_ref=120)
ev1 = EvidentlyMonitoringStep(data_drift_test_suite, d1, logger_factory, min_instances=240, clock=120, name='data_drift_eval')

# Detector 1.2 - Target Drift
target_drift = TestSuite(tests=[
    TestColumnDrift(column_name='target'),
])
d2 = DummyMonitoringStrategy(n_curr=120, n_ref=120, type='target')
ev2 = EvidentlyMonitoringStep(target_drift, d2, logger_factory, min_instances=240, clock=120, name='target_drift_eval')

input_monitoring = ModelMonitoringPipeline([(ev1._name, ev1), (ev2._name, ev2)])
input_drift_retraining_strategy = DummyRetrainingStrategy(n_last_retrain=120, n_last_test=0)
input_detector = DriftDetector(input_monitoring, input_drift_retraining_strategy)
###




### OUTPUT (PERFORMANCE) DRIFT DETECTION
# Detector 2.1 - Performance Drift

performance_drift = TestSuite(tests=[
    TestPrecisionScore(),
    TestRecallScore(),
    TestF1Score(),
    TestAccuracyScore()
])
d3 = DummyMonitoringStrategy(n_curr=120, n_ref=120, type='prediction')
ev3 = EvidentlyMonitoringStep(performance_drift, d3, logger_factory, min_instances=360, clock=120, name='performance_drift_eval')

output_monitoring = ModelMonitoringPipeline([(ev3._name, ev3)])
output_drift_retraining_strategy = DummyRetrainingStrategy(n_last_retrain=120, n_last_test=0)
output_detector = DriftDetector(output_monitoring, output_drift_retraining_strategy)
###

### Models comparison (after retraining)
#model_comparer = BatchModelComparer()
model_comparer = ShadowOnlineComparer(n_online=20)
###


### Model's Performance Evaluation
acc = Accuracy()
roc_auc = ROCAUC()
eval_pipe = ModelEvaluationPipeline(metric_steps=[
    ('accuracy', acc),
    ('roc_auc', roc_auc)
])
###


### Model composition
sklearn_batch_classifier = SklearnEstimator(Pipeline([('rf', RandomForestClassifier())]))
batch_pipeline = BatchPipeline(
    sklearn_batch_classifier,
    input_drift_detector=input_detector,
    output_drift_detector=output_detector,
    history=history,
    logger_factory=logger_factory,
    model_comparer=model_comparer,
    min_samples_retrain=120,
    min_samples_first_fit=240
)
###

### Experiment
experiment = StreamExperiment(batch_pipeline, eval_pipe, logger_factory)


In [2]:
X, Y = load_breast_cancer(return_X_y=True)
df = pd.DataFrame(X)
df['target'] = Y


In [3]:
experiment.run(df)

INFO:test-2220_data_drift_eval:Drift detected at: 360.
INFO:test-2220_BatchPipeline:Iter=360: drift detected. in detector num.: 0
INFO:test-2220_BatchPipeline:Iter=360: Comparing the old and the retrained model.
INFO:test-2220_BatchPipeline:Iter=381: online model selection: replacing model with the retrained one.
INFO:test-2220_performance_eval:Logging a batch of 100 reports.
INFO:test-2220_data_drift_eval:Drift detected at: 480.
INFO:test-2220_BatchPipeline:Iter=480: drift detected. in detector num.: 0
INFO:test-2220_BatchPipeline:Iter=480: Comparing the old and the retrained model.
INFO:test-2220_performance_eval:Logging a batch of 100 reports.
INFO:test-2220_BatchPipeline:Iter=501: online model selection: replacing model with the retrained one.
INFO:test-2220_performance_eval:Logging the last batch of 69 reports.


In [11]:
from river.metrics import Accuracy
from river import utils

Y_true =    [1, 0, 1, 0, 1, 0, 0]
Y_predict = [1, 1, 0, 0, 1, 1, 1]


roll_acc = utils.Rolling(Accuracy(), window_size=3)
for y_t, y_p in zip(Y_true, Y_predict):
    print(roll_acc.update(y_t, y_p).get())

1.0
0.5
0.3333333333333333
0.3333333333333333
0.6666666666666666
0.6666666666666666
0.3333333333333333


In [ ]:
from batchstream.pipelines.base.stream_pipeline import StreamPipeline
from batchstream.utils.logging.base.logger_factory import LoggerFactory


class CombinationPipeline(StreamPipeline):

    def __init__(self, members: List[StreamPipeline], combiner: object):
        self._members = members
        self._combiner = combiner

    def handle(self, x, y) -> int:
        return self._combiner.combine(x, y, self._members)



In [ ]:
import pandas as pd
from evidently.test_suite import TestSuite
from evidently.test_preset import DataDriftTestPreset
from evidently.tests import *
from batchstream.history.base.history_manager import HistoryManager
from batchstream.monitoring.pipeline.model_monitoring_pipeline import ModelMonitoringPipeline
from batchstream.monitoring.pipeline.steps.batch.evidently_monitoring_step import EvidentlyMonitoringStep
from sklearn.datasets import load_breast_cancer
from batchstream.utils.logging.base.logger_factory import LoggerFactory
from batchstream.batch_monitoring_strategy.simple_monitoring_strategy import SimpleMonitoringStrategy
from batchstream.retraining_strategy.simple_retraining_strategy import SimpleRetrainingStrategy 
from batchstream.model_comparers.batch_comparer import BatchModelComparer
from batchstream.model_comparers.shadow_comparer import ShadowOnlineComparer
from batchstream.pipelines.batch.batch_pipeline import BatchPipeline
from batchstream.estimators.sklearn_estimator import SklearnEstimator
from batchstream.detectors.base.drift_handler import DriftHandler
from batchstream.experiment.experiment import StreamExperiment
from river.metrics import Accuracy, ROCAUC
from batchstream.evaluation.river_evaluation_pipeline import RiverEvaluationPipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline



logger_factory = LoggerFactory('test-2218')
retraining_strategy = SimpleRetrainingStrategy(
  n_last_retrain=500, n_last_test=0)


### INPUT DRIFT DETECTION
# Detector 1.1 - Data Drift
data_drift_test_suite = TestSuite(tests=[
DataDriftTestPreset(),
])
d1 = SimpleMonitoringStrategy(n_curr=120, n_ref=120)
ev1 = EvidentlyMonitoringStep(data_drift_test_suite, d1, logger_factory, min_instances=240, clock=120, name='data_drift_eval')

# Detector 1.2 - Target Drift
target_drift = TestSuite(tests=[
    TestColumnDrift(column_name='target'),
])
d2 = SimpleMonitoringStrategy(n_curr=120, n_ref=120, type='target')
ev2 = EvidentlyMonitoringStep(target_drift, d2, logger_factory, min_instances=240, clock=120, name='target_drift_eval')

input_monitoring = ModelMonitoringPipeline([(ev1._name, ev1), (ev2._name, ev2)])
input_detector = DriftHandler(input_monitoring, retraining_strategy)
###




### OUTPUT (PERFORMANCE) DRIFT DETECTION
# Detector 2.1 - Performance Drift

performance_drift = TestSuite(tests=[
    TestPrecisionScore(),
    TestRecallScore(),
    TestF1Score(),
    TestAccuracyScore()
])
d3 = SimpleMonitoringStrategy(n_curr=500, n_ref=500, type='prediction')
ev3 = EvidentlyMonitoringStep(performance_drift, d3, logger_factory,
  min_instances=1000, clock=500, name='performance_drift_eval')

# Output Drift Handler (Performance Drift + Retraining Strategy)
output_monitoring = ModelMonitoringPipeline([(ev3._name, ev3)])

output_drift_handlers = [
  DriftHandler(output_monitoring, retraining_strategy)
]
###

### Models comparison (after retraining)
#model_comparer = BatchModelComparer()
model_comparer = ShadowOnlineComparer(n_online=100)
###


### Model's Performance Evaluation
acc = Accuracy()
roc_auc = ROCAUC()
eval_pipe = RiverEvaluationPipeline(metric_steps=[
    ('accuracy', acc),
    ('roc_auc', roc_auc)
])
###


### Model composition
logger_factory = LoggerFactory(experiment_id='rf_exp')
Pipeline([('rf', RandomForestClassifier(max_depth=10))])
sklearn_batch_classifier = SklearnEstimator()
batch_pipeline = BatchPipeline(
    sklearn_batch_classifier,
    input_drift_handlers,
    output_drift_handlers,
    history,
    logger_factory,
    model_comparer,
    min_samples_retrain=500,
    min_samples_first_fit=1000
)

In [ ]:
from river.metrics import Accuracy, MacroF1, MicroF1
from river.utils import Rolling



window_size = 1000
eval_pipe = RiverEvaluationPipeline(metric_steps=[
  (f'acc_preq_{window_size}', Rolling(Accuracy(), window_size)),
  (f'micro_f1_preq_{window_size}', Rolling(MicroF1(), window_size)),
  (f'macro_f1_preq_{window_size}', Rolling(MacroF1(), window_size))
])

In [2]:
from batchstream.pipelines.stream.model_river_pipeline import RiverPipeline


In [ ]:
from river.forest import ARFClassifier
from river import ensemble
from river import evaluate
from river import metrics
from river.datasets import synth
from river import tree
from river import ADWIN
from river import naive_bayes


In [3]:
from river.ensemble import SRPClassifier

ImportError: cannot import name 'SRPClassifier' from 'river' (c:\Users\golik\.conda\envs\test\lib\site-packages\river\__init__.py)

In [ ]:
from river.forest import ARFClassifier


logger_factory = LoggerFactory(experiment_id='arf_exp')
arf_model = ARFClassifier(seed=42, leaf_prediction="mc")
arf_pipe = RiverPipeline(arf_model)
arf_experiment = StreamExperiment(arf_pipe, eval_pipe, logger_factory)

In [ ]:
base_model = tree.HoeffdingTreeClassifier(grace_period=50, delta=0.01, nominal_attributes=['age', 'car', 'zipcode'])
srp_model = ensemble.SRPClassifier(model=base_model, n_models=3, seed=42)
srp_pipe = RiverPipeline(srp_model)

In [ ]:
from river.tree import HoeffdingAdaptiveTreeClassifier



hat_model = HoeffdingAdaptiveTreeClassifier(
  grace_period=100,
  delta=1e-5,
  leaf_prediction='nb',
  nb_threshold=10,
  seed=42
)
hat_pipe = RiverPipeline(hat_model)

In [ ]:
from river import naive_bayes



logger_factory = LoggerFactory(experiment_id='nb_exp')
nb_model =  naive_bayes.GaussianNB()
nb_pipe = RiverPipeline(nb_model)
nb_experiment = StreamExperiment(nb_pipe, eval_pipe, logger_factory)

In [ ]:
from evidently.test_suite import TestSuite
from evidently.test_preset import DataDriftTestPreset
from evidently.tests import *



### INPUT DRIFT DETECTION
# Detector 1.1 - Data Drift
data_drift_test_suite = TestSuite(tests=[
  DataDriftTestPreset(),
])
d1 = DummyMonitoringStrategy(n_curr=500, n_ref=500, type='data')
ev1 = EvidentlyMonitoringStep(data_drift_test_suite, d1, logger_factory,
  min_instances=1000, clock=500, name='data_drift_eval'
)

# Detector 1.2 - Target Drift 
target_drift = TestSuite(tests=[
    TestColumnDrift(column_name='target'),
])
d2 = DummyMonitoringStrategy(n_curr=500, n_ref=500, type='target')
ev2 = EvidentlyMonitoringStep(target_drift, d2, logger_factory,
  min_instances=1000, clock=500, name='target_drift_eval'
)

# Input Drift Handler (Data + Target Drift + Retraining Strategy)
input_monitoring = ModelMonitoringPipeline([(ev1._name, ev1), (ev2._name, ev2)])
input_drift_retraining_strategy = DummyRetrainingStrategy(
  n_last_retrain=500, n_last_test=0
)
input_drift_handlers = [
  DriftHandler(input_monitoring, input_drift_retraining_strategy)
]


In [ ]:
StreamExperiment()

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline



logger_factory = LoggerFactory(experiment_id='rf_exp_all_drifts')
rf_model = Pipeline([('rf', RandomForestClassifier(max_depth=10))])
sklearn_batch_classifier = SklearnEstimator(rf_model)
batch_pipeline = BatchPipeline(
    sklearn_batch_classifier,
    input_drift_handlers,
    output_drift_handlers,
    history,
    logger_factory,
    model_comparer,
    min_samples_retrain=1500,
    min_samples_first_fit=1000
)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline



logger_factory = LoggerFactory(experiment_id='lr_exp_all_drifts')
lr_model = Pipeline([('lr', LogisticRegression())])
sklearn_batch_classifier = SklearnEstimator(lr_model)
batch_pipeline = BatchPipeline(
    sklearn_batch_classifier,
    input_drift_handlers,
    output_drift_handlers,
    history,
    logger_factory,
    model_comparer,
    min_samples_retrain=1500,
    min_samples_first_fit=1000
)

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import Pipeline



logger_factory = LoggerFactory(experiment_id='nb_exp_all_drifts')
nb_model = Pipeline([('nb', GaussianNB())])
sklearn_batch_classifier = SklearnEstimator(nb_model)
batch_pipeline = BatchPipeline(
    sklearn_batch_classifier,
    input_drift_handlers,
    output_drift_handlers,
    history,
    logger_factory,
    model_comparer,
    min_samples_retrain=1500,
    min_samples_first_fit=1000
)

In [ ]:
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline



logger_factory = LoggerFactory(experiment_id='xgb_exp_all_drifts')
xgb_model = Pipeline([('xgb', XGBClassifier())])
sklearn_batch_classifier = SklearnEstimator(xgb_model)
batch_pipeline = BatchPipeline(
    sklearn_batch_classifier,
    input_drift_handlers,
    output_drift_handlers,
    history,
    logger_factory,
    model_comparer,
    min_samples_retrain=1500,
    min_samples_first_fit=1000
)

In [2]:
!pip install xgboost

     ---------------------------------------- 70.9/70.9 MB 1.1 MB/s eta 0:00:00
